In [6]:
from backend import ercot_df

ercot_df.head()

100%|██████████| 3/3 [00:00<00:00,  3.39it/s]


,Oper Day,Interval Ending,HB_BUSAVG,HB_HOUSTON,HB_HUBAVG,HB_NORTH,HB_PAN,HB_SOUTH,HB_WEST,LZ_AEN,LZ_CPS,LZ_HOUSTON,LZ_LCRA,LZ_NORTH,LZ_RAYBN,LZ_SOUTH,LZ_WEST
0,11/11/2024,0015,32.02,22.83,32.35,34.05,43.46,22.34,50.17,19.97,19.90,22.64,16.11,34.02,34.46,32.01,71.27
1,11/11/2024,0030,31.74,24.19,32.72,32.64,41.65,23.22,50.85,21.05,21.26,24.02,17.76,32.62,32.95,30.18,85.75
2,11/11/2024,0045,30.11,23.26,30.52,31.60,39.19,22.04,45.18,20.34,20.09,23.08,17.33,31.57,31.92,28.60,65.23
3,11/11/2024,0100,29.88,23.10,30.37,31.22,38.79,22.07,45.09,20.32,20.19,22.92,17.40,31.20,31.52,28.63,66.80
4,11/11/2024,0115,26.20,21.57,27.43,25.86,32.10,21.54,40.76,19.85,20.59,21.49,18.04,25.81,25.99,25.62,76.85


In [7]:
import pandas as pd

ercot_df = ercot_df.apply(pd.to_numeric, errors='coerce')
ercot_df.rename(columns={'Oper Day':'Oper_Day', 'Interval Ending': 'Interval_Ending'}, inplace=True)

In [8]:
from tovowpal import df_to_vw

vw = df_to_vw(ercot_df, label='HB_BUSAVG')
for i in range(3):
    print(vw[i] + '\n')

<class 'list'>
Oper_Day           float64
Interval_Ending      int64
HB_BUSAVG          float64
HB_HOUSTON         float64
HB_HUBAVG          float64
HB_NORTH           float64
HB_PAN             float64
HB_SOUTH           float64
HB_WEST            float64
LZ_AEN             float64
LZ_CPS             float64
LZ_HOUSTON         float64
LZ_LCRA            float64
LZ_NORTH           float64
LZ_RAYBN           float64
LZ_SOUTH           float64
LZ_WEST            float64
dtype: object
32.02 |  Interval_Ending:15 HB_HOUSTON:22.83 HB_HUBAVG:32.35 HB_NORTH:34.05 HB_PAN:43.46 HB_SOUTH:22.34 HB_WEST:50.17 LZ_AEN:19.97 LZ_CPS:19.9 LZ_HOUSTON:22.64 LZ_LCRA:16.11 LZ_NORTH:34.02 LZ_RAYBN:34.46 LZ_SOUTH:32.01 LZ_WEST:71.27

31.74 |  Interval_Ending:30 HB_HOUSTON:24.19 HB_HUBAVG:32.72 HB_NORTH:32.64 HB_PAN:41.65 HB_SOUTH:23.22 HB_WEST:50.85 LZ_AEN:21.05 LZ_CPS:21.26 LZ_HOUSTON:24.02 LZ_LCRA:17.76 LZ_NORTH:32.62 LZ_RAYBN:32.95 LZ_SOUTH:30.18 LZ_WEST:85.75

30.11 |  Interval_Ending:45 HB_HOUSTON:23.2

In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(vw, test_size=0.2)

# Model

In [10]:
from vowpalwabbit import Workspace

model = Workspace('--loss_function squared')

for row in train:
    model.learn(row)
    
predictions = []

for row in test:
    predictions.append(model.predict(row))
    
model.finish()

In [11]:
print(predictions[:5])

[27.456188201904297, 19.469083786010742, 17.36369514465332, 30.446422576904297, 28.819578170776367]
